In [1]:
import logging
from random import randint
import random
import errno
import functools
from loguru import logger
import signal
import time
import json
import faiss
import subprocess
import shutil
import psutil
import pickle
import heapq
from collections import defaultdict
from datetime import datetime

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

import lean_dojo
from utils.lean_math_utils import *

In [2]:
import os
from lean_dojo import *
import random

print("Imports done")

# fin = open('/home/mcwave/code/automath/atp/datasets/traced_repo_math_in_lean.pkl', 'rb')
# traced_repo = pickle.load(fin)
# fin.close()
# print("Loaded traced repo")

github_token = os.getenv("GITHUB_ACCESS_TOKEN", None)
# print("token:", github_token)
repo = LeanGitRepo(
#     "https://github.com/leanprover-community/mathematics_in_lean",
#     "210802794074c003ddf237ee56a0bf1fb5a9fe10"
    "https://github.com/yangky11/miniF2F-lean4",
    "6bcf0b4940fbf17a1ba83db4ed639fbcb26b1a27"
)

Imports done


In [3]:
tokenizer_name = "morph-labs/morph-prover-v0-7b" #"internlm/internlm2-math-7b" #"ScalableMath/Lean-STaR-plus"  # 'Saisam/gpt-neo-math-small' #

tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
tokenizer.padding_side = "right"

# Define the separator token
sep_token = "<sep>"
pad_token = "<pad>"
eos_token = "<end>"

# Check if the separator token already exists in the vocabulary
if sep_token not in tokenizer.get_vocab():
    tokenizer.add_tokens([sep_token])
if pad_token not in tokenizer.get_vocab():
    tokenizer.add_tokens([pad_token])

# Set the separator token
tokenizer.sep_token = sep_token
tokenizer.pad_token = pad_token

tokenizer.add_special_tokens({
    'sep_token': sep_token,
    'pad_token': pad_token,
    'eos_token': eos_token
})

3

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import PreTrainedTokenizerFast, AutoModelForCausalLM, AutoModelForSeq2SeqLM, AdamW
from tqdm import tqdm
import os

class InstructionResponseDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        instruction, response = self.data[idx]
        
        input_encoding = self.tokenizer(instruction, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")
        output_encoding = self.tokenizer(response, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")

        input_mask = input_encoding["attention_mask"].squeeze()

        return {
            "input_ids": input_encoding["input_ids"].squeeze(),
            "attention_mask": input_mask,
            "labels": output_encoding["input_ids"].squeeze()
        }

def evaluate(model, dataloader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

    return total_loss / len(dataloader)

# Initialize tokenizer and model
# model_name = "google/byt5-small"  # You can change this to any other Seq2Seq model
model_name = '/home/mcwave/code/automath/atp/models/flan_base/trained_model_700000'

# model = AutoModelForSeq2SeqLM.from_pretrained(
#     model_name,
#     device_map="auto",
#     torch_dtype=torch.bfloat16,
# )
model = torch.load(model_name)

# Set up optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# model.eval()

/tmp/ipykernel_19845/570655300.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_name)
/home/mcwave/anaconda3/envs/atp/lib/python3.10/site-packag

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [5]:
from openai import OpenAI
import os
os.environ['OPENAI_API_KEY'] = 'insert api key'

messages = []
client = OpenAI(
    # This is the default and can be omitted
    api_key="insert api key",
)

def generate_tac_gpt(state):
    chat = client.chat.completions.create(
            model="gpt-4o", messages=[{"role": "user", "content": f"Generate a Lean 4 proof for the following state: {state}. Say nothing, except for the tactic."}]
        )
    
    reply = chat.choices[0].message.content
    return reply

In [6]:
def generate_response(instruction):
    input_ids = tokenizer(instruction, return_tensors="pt").input_ids.to(device)
#     print("ids", input_ids)
    output = model.generate(input_ids, num_beams=4, length_penalty=0, do_sample = True, eos_token_id = tokenizer.eos_token_id)
#     print("out", output)
    return [tokenizer.decode(tac, skip_special_tokens=False) for tac in output]

def generate_tac(instruction):
#     print("inst", instruction)
    res = generate_response(instruction)
#     print("res", res)
    return res[0].split("<s> ")[1].split("<pad>")[0]

test_instruction = """R : Type u_1\ninst✝ : Ring R\na : R\n⊢ a + 0 = a"""
response = generate_tac(test_instruction)
print(f"Instruction: {test_instruction}")
print(f"Response 1: {response}")

Instruction: R : Type u_1
inst✝ : Ring R
a : R
⊢ a + 0 = a
Response 1: rw [← add_comm]


/home/mcwave/anaconda3/envs/atp/lib/python3.10/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [7]:
def get_proof(file_path, full_name):
    theorem = Theorem(repo, file_path, full_name)
    dojo, state_0 = Dojo(theorem).__enter__()
    curr_state = state_0
    pp_to_state = {}
    seen = set()
    beam = [(explore_state_complexity(curr_state.pp), curr_state.pp)]
    pp_to_state[curr_state.pp] = curr_state
    seen.add(curr_state.pp)
    beam_width = 2
    proven = False
    start_time = time.time()
    states_generated = []
    num_fails = 0
    while beam:
        new_beam = []
        
        for complexity, state in beam[:beam_width]:
            seen.add(state)
#             print("state", state)
#             print()
            state = pp_to_state[state]
            states_generated.append(time.time() - start_time)
            if type(state) == lean_dojo.interaction.dojo.ProofFinished:
                return True
            
            candidates = generate_response(state.pp)
            
            for tac in candidates:
                if time.time() - start_time > 120:
#                     print("Out of time")
                    return False
                new_state = dojo.run_tac(state, tac)
#                 print(tac)
                if type(new_state) in [LeanError,TimeoutError,TacticResult,DojoCrashError,DojoHardTimeoutError,DojoInitError,ProofGivenUp]:
#                     print("Error\n")
                    continue
                elif type(new_state) == lean_dojo.interaction.dojo.ProofFinished:
#                     print("Found proof\n")
                    return True, states_generated
                if new_state.pp in seen:
#                     print("Seen\n")
                    continue
#                 print("Successful\n")
                complexity = explore_state_complexity(new_state.pp, complexity)
                pp_to_state[new_state.pp] = new_state
                new_beam.append((complexity, new_state.pp))
        if not new_beam and num_fails < 10:
#             print("Failed beam search, trying again")
            num_fails += 1
            continue
        beam = heapq.nsmallest(beam_width, new_beam)
        
        if proven:
            return True, states_generated
        
        if not beam:
            print("Out of beams")
            return False, states_generated
    return False, states_generated

def explore_state_complexity(state, base_complexity=None, seen_target_freq=None):
    if '⊢ False' in state:
        return 1000000
    if base_complexity is not None:
        return base_complexity + 1
    complexity = 0
    lines = state.split('\n')
    targets = []
    min_freq = 1000000
    for line in lines:
        if line.startswith("⊢"):
            target = line[1:].strip()
            targets.append(target)
            if seen_target_freq is not None:
                if target in seen_target_freq:
                    min_freq = min(min_freq, seen_target_freq[target])
                    seen_target_freq[target] = seen_target_freq[target] + 1
                else:
                    min_freq = 0
                    seen_target_freq[target] = 1
    lengths = [len(x) for x in targets]
    complexity = max(lengths)/2 + sum(lengths)/2
    if seen_target_freq is not None:
        complexity += PENALTY_SEEN_TARGET_MULTIPLIER*min_freq
    return complexity

In [8]:
import signal

class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException("Function call timed out")

# Register the signal function handler
signal.signal(signal.SIGALRM, timeout_handler)

<Handlers.SIG_DFL: 0>

In [9]:
def get_proof_nobeam(file_path, full_name):
    theorem = Theorem(repo, file_path, full_name)
    dojo, state_0 = Dojo(theorem).__enter__()
    curr_state = state_0
    proven = False
    start_time = time.time()
    seen = set()
    num_tries = 0
    
    signal.alarm(120)
#     print("Started")
    try:
        while num_tries < 10:
            if time.time() - start_time > 120:
                return False
            start_time = time.time()
            try:
#                 print("Generating Tactic")
#                 tac = generate_tac(curr_state.pp)
                tac = generate_tac_gpt(curr_state.pp)#.replace("`", "").replace("lean", "").replace("\n", "")
#                 print("Transition Time", time.time() - start_time)
#                 print(curr_state)
#                 print(tac)
#                 print(seen)
                new_state = dojo.run_tac(curr_state, tac)
#                 print("Finished running tactic")
            except:
                num_tries += 1
                continue
            if type(new_state) in [LeanError,TimeoutError,TacticResult,DojoCrashError,DojoHardTimeoutError,DojoInitError,ProofGivenUp]:
                num_tries += 1
                continue
            elif type(new_state) == lean_dojo.interaction.dojo.ProofFinished:
                return True

            if new_state.pp in seen:
                num_tries += 1
                continue
            seen.add(new_state.pp)
            curr_state = new_state
            num_tries = 0

        return False
    except TimeoutException:
        print("Out of time")
        return False
    finally:
        signal.alarm(0)

In [10]:
# dojo, state_0 = Dojo(Theorem(repo, "MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyGroup.mul_one")).__enter__()

In [11]:
# state_0

In [12]:
# state_1 = dojo.run_tac(state_0, "exact mul_one a")
# state_1

In [13]:
# state_2 = dojo.run_tac(state_1, "rw [← mul_assoc]")
# state_2

In [14]:
# state_3 = dojo.run_tac(state_2, "rw [mul_right_inv]")
# state_3

In [15]:
def get_thms(file_path):
    theorems_list = []
    f = open(file_path, "r")
    all_thms = f.read().split("theorem ")[1:]
    real_path = "/".join(file_path.split("/")[7:])
    for thm in all_thms:
        theorems_list.append((real_path, thm.split(" ")[0].replace("\n", "")))
    return theorems_list

In [16]:
theorems_list = get_thms("/home/mcwave/code/automath/atp/miniF2F-lean4/MiniF2F/Test.lean") + get_thms("/home/mcwave/code/automath/atp/miniF2F-lean4/MiniF2F/Valid.lean")
theorems_list

[('MiniF2F/Test.lean', 'mathd_algebra_478'),
 ('MiniF2F/Test.lean', 'numbertheory_4x3m7y3neq2003'),
 ('MiniF2F/Test.lean', 'aime_1983_p1'),
 ('MiniF2F/Test.lean', 'amc12_2001_p5'),
 ('MiniF2F/Test.lean', 'mathd_algebra_141'),
 ('MiniF2F/Test.lean', 'mathd_numbertheory_3'),
 ('MiniF2F/Test.lean', 'imo_1969_p2'),
 ('MiniF2F/Test.lean', 'mathd_algebra_44'),
 ('MiniF2F/Test.lean', 'mathd_algebra_209'),
 ('MiniF2F/Test.lean', 'mathd_numbertheory_1124'),
 ('MiniF2F/Test.lean', 'imo_1983_p6'),
 ('MiniF2F/Test.lean', 'mathd_numbertheory_237'),
 ('MiniF2F/Test.lean', 'mathd_algebra_33'),
 ('MiniF2F/Test.lean', 'amc12b_2021_p3'),
 ('MiniF2F/Test.lean', 'mathd_numbertheory_299'),
 ('MiniF2F/Test.lean', 'amc12b_2020_p2'),
 ('MiniF2F/Test.lean', 'algebra_sqineq_unitcircatbpabsamblt1'),
 ('MiniF2F/Test.lean', 'imo_1977_p6'),
 ('MiniF2F/Test.lean', 'mathd_algebra_419'),
 ('MiniF2F/Test.lean', 'amc12a_2020_p10'),
 ('MiniF2F/Test.lean', 'imo_1960_p2'),
 ('MiniF2F/Test.lean', 'mathd_numbertheory_427'),


In [17]:
# theorems_list = [("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.add_neg_cancel_right"),
# ("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.add_left_cancel"),
# ("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.add_right_cancel"),
# ("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.zero_mul"),
# ("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.neg_eq_of_add_eq_zero"),
# ("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.eq_neg_of_add_eq_zero"),
# ("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.neg_zero"),
# ("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.neg_neg"),
# ("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.self_sub"),
# ("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.one_add_one_eq_two"),
# ("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.two_mul"),
# ("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyGroup.mul_right_inv"),
# ("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyGroup.mul_one"),
# ("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyGroup.mul_inv_rev"),
# ("MIL/C02_Basics/S05_Proving_Facts_about_Algebraic_Structures.lean", "absorb1"),
# ("MIL/C02_Basics/S05_Proving_Facts_about_Algebraic_Structures.lean", "absorb2"),
# ("MIL/C02_Basics/S05_Proving_Facts_about_Algebraic_Structures.lean", "aux1"),
# ("MIL/C02_Basics/S05_Proving_Facts_about_Algebraic_Structures.lean", "aux2"),
# ("MIL/C02_Basics/S03_Using_Theorems_and_Lemmas.lean", "fact1"),
# ("MIL/C02_Basics/S03_Using_Theorems_and_Lemmas.lean", "fact2"),
# ("MIL/C02_Basics/S04_More_on_Order_and_Divisibility.lean", "C02S04.aux"),
# ("MIL/C03_Logic/S05_Disjunction.lean", "C03S05.MyAbs.le_abs_self"),
# ("MIL/C03_Logic/S05_Disjunction.lean", "C03S05.MyAbs.neg_le_abs_self"),
# ("MIL/C03_Logic/S05_Disjunction.lean", "C03S05.MyAbs.abs_add"),
# ("MIL/C03_Logic/S05_Disjunction.lean", "C03S05.MyAbs.lt_abs"),
# ("MIL/C03_Logic/S05_Disjunction.lean", "C03S05.MyAbs.abs_lt"),
# ("MIL/C03_Logic/S04_Conjunction_and_Iff.lean", "C03S04.aux"),
# ("MIL/C03_Logic/S04_Conjunction_and_Iff.lean", "C03S04.not_monotone_iff"),
# ("MIL/C03_Logic/S01_Implication_and_the_Universal_Quantifier.lean", "C03S01.my_lemma4"),
# ("MIL/C03_Logic/S01_Implication_and_the_Universal_Quantifier.lean", "C03S01.FnUb"),
# ("MIL/C03_Logic/S01_Implication_and_the_Universal_Quantifier.lean", "C03S01.FnLb"),
# ("MIL/C03_Logic/S01_Implication_and_the_Universal_Quantifier.lean", "C03S01.FnEven"),
# ("MIL/C03_Logic/S01_Implication_and_the_Universal_Quantifier.lean", "C03S01.FnOdd"),
# ("MIL/C03_Logic/S01_Implication_and_the_Universal_Quantifier.lean", "C03S01.Subset.trans"),
# ("MIL/C03_Logic/S01_Implication_and_the_Universal_Quantifier.lean", "C03S01.SetUb"),
# ("MIL/C03_Logic/S06_Sequences_and_Convergence.lean", "C03S06.ConvergesTo"),
# ("MIL/C03_Logic/S06_Sequences_and_Convergence.lean", "C03S06.convergesTo_const"),
# ("MIL/C03_Logic/S06_Sequences_and_Convergence.lean", "C03S06.convergesTo_add"),
# ("MIL/C03_Logic/S06_Sequences_and_Convergence.lean", "C03S06.convergesTo_mul_const"),
# ("MIL/C03_Logic/S06_Sequences_and_Convergence.lean", "C03S06.exists_abs_le_of_convergesTo"),
# ("MIL/C03_Logic/S06_Sequences_and_Convergence.lean", "C03S06.aux"),
# ("MIL/C03_Logic/S06_Sequences_and_Convergence.lean", "C03S06.convergesTo_mul"),
# ("MIL/C03_Logic/S06_Sequences_and_Convergence.lean", "C03S06.convergesTo_unique"),
# ("MIL/C03_Logic/S02_The_Existential_Quantifier.lean", "C03S02.FnUb"),
# ("MIL/C03_Logic/S02_The_Existential_Quantifier.lean", "C03S02.FnLb"),
# ("MIL/C03_Logic/S02_The_Existential_Quantifier.lean", "C03S02.FnHasUb"),
# ("MIL/C03_Logic/S02_The_Existential_Quantifier.lean", "C03S02.FnHasLb"),
# ("MIL/C03_Logic/S02_The_Existential_Quantifier.lean", "C03S02.fnUb_add"),
# ("MIL/C03_Logic/S03_Negation.lean", "C03S03.FnUb"),
# ("MIL/C03_Logic/S03_Negation.lean", "C03S03.FnLb"),
# ("MIL/C03_Logic/S03_Negation.lean", "C03S03.FnHasUb"),
# ("MIL/C03_Logic/S03_Negation.lean", "C03S03.FnHasLb"),
# ("MIL/C04_Sets_and_Functions/S02_Functions.lean", "inverse"),
# ("MIL/C04_Sets_and_Functions/S02_Functions.lean", "inverse_spec"),
# ("MIL/C04_Sets_and_Functions/S02_Functions.lean", "Cantor"),
# ("MIL/C04_Sets_and_Functions/S01_Sets.lean", "primes"),
# ("MIL/C04_Sets_and_Functions/S03_The_Schroeder_Bernstein_Theorem.lean", "sbAux"),
# ("MIL/C04_Sets_and_Functions/S03_The_Schroeder_Bernstein_Theorem.lean", "sbSet"),
# ("MIL/C04_Sets_and_Functions/S03_The_Schroeder_Bernstein_Theorem.lean", "sbFun"),
# ("MIL/C04_Sets_and_Functions/S03_The_Schroeder_Bernstein_Theorem.lean", "sb_right_inv"),
# ("MIL/C04_Sets_and_Functions/S03_The_Schroeder_Bernstein_Theorem.lean", "sb_injective"),
# ("MIL/C04_Sets_and_Functions/S03_The_Schroeder_Bernstein_Theorem.lean", "sb_surjective"),
# ("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "fac"),
# ("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "pow_two_le_fac"),
# ("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "sum_sqr"),
# ("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat"),
# ("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.add"),
# ("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.mul"),
# ("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.zero_add"),
# ("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.succ_add"),
# ("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.add_comm"),
# ("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.add_assoc"),
# ("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.mul_add"),
# ("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.zero_mul"),
# ("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.succ_mul"),
# ("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.mul_comm"),
# ("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.two_le"),
# ("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.exists_prime_factor"),
# ("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.primes_infinite"),
# ("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "Nat.Prime.eq_of_dvd_of_prime"),
# ("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.mem_of_dvd_prod_primes"),
# ("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.primes_infinite'"),
# ("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.bounded_of_ex_finset"),
# ("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.ex_finset_of_bounded"),
# ("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.mod_4_eq_3_or_mod_4_eq_3"),
# ("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.two_le_of_mod_4_eq_3"),
# ("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.aux"),
# ("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.exists_prime_factor_mod_4_eq_3"),
# ("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.primes_mod_4_eq_3_infinite"),
# ("MIL/C05_Elementary_Number_Theory/S01_Irrational_Roots.lean", "even_of_even_sqr"),
# ("MIL/C05_Elementary_Number_Theory/S01_Irrational_Roots.lean", "factorization_mul'"),
# ("MIL/C05_Elementary_Number_Theory/S01_Irrational_Roots.lean", "factorization_pow'"),
# ("MIL/C05_Elementary_Number_Theory/S01_Irrational_Roots.lean", "Nat.Prime.factorization'"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.zero_def"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.one_def"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.add_def"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.neg_def"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.mul_def"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.zero_re"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.zero_im"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.one_re"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.one_im"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.add_re"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.add_im"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.neg_re"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.neg_im"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.mul_re"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.mul_im"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.instCommRing"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.sub_re"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.sub_im"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "Int.div'"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "Int.mod'"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "Int.div'_add_mod'"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "Int.abs_mod'_le"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "Int.mod'_eq"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "aux"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "sq_add_sq_eq_zero"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.norm"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.norm_nonneg"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.norm_eq_zero"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.norm_pos"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.norm_mul"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.conj"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.conj_re"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.conj_im"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.norm_conj"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.div_def"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.mod_def"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.norm_mod_lt"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.coe_natAbs_norm"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.natAbs_norm_mod_lt"),
# ("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.not_norm_mul_left_lt_norm"),
# ("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.AddGroup₁"),
# ("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.Point"),
# ("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.Point.add"),
# ("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.Point.neg"),
# ("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.Point.zero"),
# ("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.Point.addGroupPoint"),
# ("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.AddGroup₂"),
# ("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.hasAddAddGroup₂"),
# ("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.hasZeroAddGroup₂"),
# ("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.hasNegAddGroup₂"),
# ("MIL/C06_Structures/S01_Structures.lean", "C06S01.Point"),
# ("MIL/C06_Structures/S01_Structures.lean", "C06S01.Point.add"),
# ("MIL/C06_Structures/S01_Structures.lean", "C06S01.Point.add_assoc"),
# ("MIL/C06_Structures/S01_Structures.lean", "C06S01.Point.smul"),
# ("MIL/C06_Structures/S01_Structures.lean", "C06S01.Point.smul_distrib"),
# ("MIL/C06_Structures/S01_Structures.lean", "C06S01.StandardTwoSimplex"),
# ("MIL/C06_Structures/S01_Structures.lean", "C06S01.StandardTwoSimplex.weightedAverage"),
# ("MIL/C06_Structures/S01_Structures.lean", "C06S01.StandardSimplex"),
# ("MIL/C06_Structures/S01_Structures.lean", "C06S01.StandardSimplex.midpoint"),
# ("MIL/C06_Structures/S01_Structures.lean", "C06S01.StandardSimplex.weightedAverage"),
# ("MIL/C07_Hierarchies/S03_Subobjects.lean", "Submonoid₁"),
# ("MIL/C07_Hierarchies/S03_Subobjects.lean", "SubMonoid₁Monoid"),
# ("MIL/C07_Hierarchies/S03_Subobjects.lean", "SubmonoidClass₁"),
# ("MIL/C07_Hierarchies/S03_Subobjects.lean", "Subgroup₁"),
# ("MIL/C07_Hierarchies/S03_Subobjects.lean", "SubgroupClass₁"),
# ("MIL/C07_Hierarchies/S03_Subobjects.lean", "Submonoid.Setoid"),
# ("MIL/C07_Hierarchies/S03_Subobjects.lean", "QuotientMonoid.mk"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "One₁"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "One₂"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "Dia₁"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "Semigroup₁"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "Semigroup₂"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "DiaOneClass₁"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "Monoid₁"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "Monoid₂"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "Inv₁"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "Group₁"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "left_inv_eq_right_inv₁"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "inv_eq_of_dia"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "dia_inv"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "AddSemigroup₃"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "Semigroup₃"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "AddMonoid₃"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "Monoid₃"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "left_inv_eq_right_inv'"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "AddCommSemigroup₃"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "CommSemigroup₃"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "AddCommMonoid₃"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "CommMonoid₃"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "AddGroup₃"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "Group₃"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "inv_eq_of_mul"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "Group₃.mul_inv"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "mul_left_cancel₃"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "mul_right_cancel₃"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "AddCommGroup₃"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "CommGroup₃"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "Ring₃"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "LE₁"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "Preorder₁"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "PartialOrder₁"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "OrderedCommMonoid₁"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "SMul₃"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "Module₁"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "selfModule"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "nsmul₁"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "zsmul₁"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "abGrpModule"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "AddMonoid₄"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "mySMul"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "LT₁"),
# ("MIL/C07_Hierarchies/S01_Basics.lean", "PreOrder₂"),
# ("MIL/C07_Hierarchies/S02_Morphisms.lean", "isMonoidHom₁"),
# ("MIL/C07_Hierarchies/S02_Morphisms.lean", "isMonoidHom₂"),
# ("MIL/C07_Hierarchies/S02_Morphisms.lean", "MonoidHom₁"),
# ("MIL/C07_Hierarchies/S02_Morphisms.lean", "AddMonoidHom₁"),
# ("MIL/C07_Hierarchies/S02_Morphisms.lean", "RingHom₁"),
# ("MIL/C07_Hierarchies/S02_Morphisms.lean", "MonoidHomClass₁"),
# ("MIL/C07_Hierarchies/S02_Morphisms.lean", "badInst"),
# ("MIL/C07_Hierarchies/S02_Morphisms.lean", "MonoidHomClass₂"),
# ("MIL/C07_Hierarchies/S02_Morphisms.lean", "map_inv_of_inv"),
# ("MIL/C07_Hierarchies/S02_Morphisms.lean", "MonoidHomClass₃"),
# ("MIL/C07_Hierarchies/S02_Morphisms.lean", "OrderPresHom"),
# ("MIL/C07_Hierarchies/S02_Morphisms.lean", "OrderPresMonoidHom"),
# ("MIL/C07_Hierarchies/S02_Morphisms.lean", "OrderPresHomClass"),
# ("MIL/C08_Groups_and_Rings/S01_Groups.lean", "conjugate"),
# ("MIL/C08_Groups_and_Rings/S01_Groups.lean", "eq_bot_iff_card"),
# ("MIL/C08_Groups_and_Rings/S01_Groups.lean", "inf_bot_of_coprime"),
# ("MIL/C08_Groups_and_Rings/S01_Groups.lean", "conjugate_one"),
# ("MIL/C08_Groups_and_Rings/S01_Groups.lean", "aux_card_eq"),
# ("MIL/C08_Groups_and_Rings/S01_Groups.lean", "iso₁"),
# ("MIL/C08_Groups_and_Rings/S01_Groups.lean", "iso₂"),
# ("MIL/C08_Groups_and_Rings/S01_Groups.lean", "finalIso"),
# ("MIL/C08_Groups_and_Rings/S02_Rings.lean", "chineseMap"),
# ("MIL/C08_Groups_and_Rings/S02_Rings.lean", "chineseMap_mk"),
# ("MIL/C08_Groups_and_Rings/S02_Rings.lean", "chineseMap_mk'"),
# ("MIL/C08_Groups_and_Rings/S02_Rings.lean", "chineseMap_inj"),
# ("MIL/C08_Groups_and_Rings/S02_Rings.lean", "isCoprime_Inf"),
# ("MIL/C08_Groups_and_Rings/S02_Rings.lean", "chineseMap_surj"),
# ("MIL/C08_Groups_and_Rings/S02_Rings.lean", "chineseIso"),
# ("MIL/C09_Topology/S03_Topological_Spaces.lean", "aux"),
# ("MIL/C09_Topology/S01_Filters.lean", "Tendsto₁"),
# ("MIL/C09_Topology/S02_Metric_Spaces.lean", """cauchySeq_of_le_geometric_two'""")]
# # theorems_list = [('MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean', 'MyRing.neg_add_cancel_left'),]

In [18]:
# # chapters = os.listdir("/home/mcwave/mathematics_in_lean/MIL")
# total_attempts = 0
# num_proven = 0
# num_proven_cats = [0] * 8
# #269K: 27/118
# #500K: 39/118
# #500K, ReProver finetuned: 45/118

# #600K: 104/493 MiniF2F
# #ReProver: 49/493 MiniF2F
# for thm_path, thm_name in theorems_list:
#     try:
#         print(thm_path, thm_name)
#         is_proven = get_proof_nobeam(thm_path, thm_name)
#         num_proven += is_proven
#     #         num_proven_cats[int(thm_path[6]) - 2] += is_proven
#         total_attempts += 1
#         state_data = {"file_path": thm_path, "full_name": thm_name, "total_attempts": total_attempts, "is_proven": is_proven}
#         with open("/home/mcwave/code/automath/atp/results/flan_base/results_700k_nondet_minif2f.json", "a") as outfile:
#             json.dump(state_data, outfile)
#             outfile.write("\n")
#         print("res", num_proven, total_attempts)
#     except Exception as e:
#         print(e)
# print(1/0)

In [21]:
model_names = ['/home/mcwave/code/automath/atp/models/flan_small/trained_model_1500000', 
              '/home/mcwave/code/automath/atp/models/flan_small/trained_model_784k',
              '/home/mcwave/code/automath/atp/models/flan_base/trained_model_640000',
              '/home/mcwave/code/automath/atp/models/flan_base/trained_model_660000',
              '/home/mcwave/code/automath/atp/models/flan_base/trained_model_680000']
model_prefix = "/home/mcwave/code/automath/atp/models/"

for model_name in model_names[:2]:
#     model = torch.load(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    optimizer = AdamW(model.parameters(), lr=5e-5)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    total_attempts = 0
    num_proven = 0
    for thm_path, thm_name in theorems_list[31:]:
        try:
            print(thm_path, thm_name)
            is_proven = get_proof_nobeam(thm_path, thm_name)
            num_proven += is_proven
            total_attempts += 1
            state_data = {"file_path": thm_path, "full_name": thm_name, "total_attempts": total_attempts, "is_proven": is_proven}
            with open(f"/home/mcwave/code/automath/atp/results/results_gpt4o_minif2f.json", "a") as outfile:
                json.dump(state_data, outfile)
                outfile.write("\n")
            print("res", num_proven, total_attempts)
        except Exception as e:
            print(e)

2024-10-19 21:44:00.748 | WARNING  | lean_dojo.interaction.dojo:__init__:156 - Using Lean 4 without a hard timeout may hang indefinitely.
2024-10-19 21:44:00.748 | INFO     | lean_dojo.interaction.dojo:__enter__:168 - Initializing Dojo for Theorem(repo=LeanGitRepo(url='https://github.com/yangky11/miniF2F-lean4', commit='6bcf0b4940fbf17a1ba83db4ed639fbcb26b1a27'), file_path=PosixPath('MiniF2F/Test.lean'), full_name='mathd_numbertheory_277')
2024-10-19 21:44:00.749 | INFO     | lean_dojo.interaction.dojo:__enter__:187 - Copy tree ...


MiniF2F/Test.lean mathd_numbertheory_277
get_traced_repo_path, path= /home/mcwave/.cache/lean_dojo/yangky11-miniF2F-lean4-6bcf0b4940fbf17a1ba83db4ed639fbcb26b1a27/miniF2F-lean4
The traced repo is available in the cache.


2024-10-19 21:44:03.317 | INFO     | lean_dojo.interaction.dojo:_modify_file:385 - Modifying MiniF2F/Test.lean
2024-10-19 21:44:03.323 | INFO     | lean_dojo.interaction.dojo:_modify_file:407 - _modify_file: proof modified
2024-10-19 21:44:03.324 | INFO     | lean_dojo.interaction.dojo:_modify_file:425 - Creating Lean4Repl.lean at Lean4Repl.lean
2024-10-19 21:44:03.324 | INFO     | lean_dojo.interaction.dojo:_modify_file:434 - _modify_file: Done writing to Lean4Repl.lean
2024-10-19 21:44:03.325 | INFO     | lean_dojo.interaction.dojo:_modify_file:439 - _modify_file: All done. Writing to /tmp/tmpep1u1zdi/miniF2F-lean4/MiniF2F/Test.lean
2024-10-19 21:44:03.325 | INFO     | lean_dojo.interaction.dojo:__enter__:207 - lake build Lean4Repl ...
2024-10-19 21:44:03.326 | INFO     | lean_dojo.interaction.dojo:__enter__:210 - Launching the proof using <class 'lean_dojo.container.NativeContainer'>
2024-10-19 21:44:03.326 | INFO     | lean_dojo.container:run:181 - Mounting[Mount(src=PosixPath('/tm

Post processing ...
Returning ...


2024-10-19 21:44:18.768 | WARNING  | lean_dojo.interaction.dojo:__init__:156 - Using Lean 4 without a hard timeout may hang indefinitely.
2024-10-19 21:44:18.768 | INFO     | lean_dojo.interaction.dojo:__enter__:168 - Initializing Dojo for Theorem(repo=LeanGitRepo(url='https://github.com/yangky11/miniF2F-lean4', commit='6bcf0b4940fbf17a1ba83db4ed639fbcb26b1a27'), file_path=PosixPath('MiniF2F/Test.lean'), full_name='mathd_numbertheory_559')
2024-10-19 21:44:18.769 | INFO     | lean_dojo.interaction.dojo:__enter__:187 - Copy tree ...


...

2024-10-20 06:14:54.401 | WARNING  | lean_dojo.interaction.dojo:__init__:156 - Using Lean 4 without a hard timeout may hang indefinitely.
2024-10-20 06:14:54.402 | INFO     | lean_dojo.interaction.dojo:__enter__:168 - Initializing Dojo for Theorem(repo=LeanGitRepo(url='https://github.com/yangky11/miniF2F-lean4', commit='6bcf0b4940fbf17a1ba83db4ed639fbcb26b1a27'), file_path=PosixPath('MiniF2F/Valid.lean'), full_name='amc12a_2002_p12')
2024-10-20 06:14:54.402 | WARNING  | lean_dojo.interaction.dojo:__init__:156 - Using Lean 4 without a hard timeout may hang indefinitely.
2024-10-20 06:14:54.402 | INFO     | lean_dojo.interaction.dojo:__enter__:168 - Initializing Dojo for Theorem(repo=LeanGitRepo(url='https://github.com/yangky11/miniF2F-lean4', commit='6bcf0b4940fbf17a1ba83db4ed639fbcb26b1a27'), file_path=PosixPath('MiniF2F/Valid.lean'), full_name='algebra_manipexpr_2erprsqpesqeqnrpnesq')
2024-10-20 06:14:54.402 | WARNING  | lean_dojo.interaction.dojo:__init__:156 - Using Lean 4 without a

2024-10-20 06:14:54.406 | WARNING  | lean_dojo.interaction.dojo:__init__:156 - Using Lean 4 without a hard timeout may hang indefinitely.
2024-10-20 06:14:54.406 | INFO     | lean_dojo.interaction.dojo:__enter__:168 - Initializing Dojo for Theorem(repo=LeanGitRepo(url='https://github.com/yangky11/miniF2F-lean4', commit='6bcf0b4940fbf17a1ba83db4ed639fbcb26b1a27'), file_path=PosixPath('MiniF2F/Valid.lean'), full_name='mathd_algebra_251')


MiniF2F/Test.lean mathd_numbertheory_277
[Errno 2] No such file or directory
MiniF2F/Test.lean mathd_numbertheory_559
[Errno 2] No such file or directory
MiniF2F/Test.lean mathd_algebra_160
[Errno 2] No such file or directory
MiniF2F/Test.lean mathd_algebra_24
[Errno 2] No such file or directory
MiniF2F/Test.lean mathd_algebra_176
[Errno 2] No such file or directory
MiniF2F/Test.lean induction_nfactltnexpnm1ngt3
[Errno 2] No such file or directory
MiniF2F/Test.lean mathd_algebra_208
[Errno 2] No such file or directory
MiniF2F/Test.lean mathd_numbertheory_353
[Errno 2] No such file or directory
MiniF2F/Test.lean numbertheory_notEquiv2i2jasqbsqdiv8
[Errno 2] No such file or directory
MiniF2F/Test.lean mathd_algebra_156
[Errno 2] No such file or directory
MiniF2F/Test.lean mathd_numbertheory_12
[Errno 2] No such file or directory
MiniF2F/Test.lean mathd_numbertheory_345
[Errno 2] No such file or directory
MiniF2F/Test.lean mathd_numbertheory_447
[Errno 2] No such file or directory
MiniF2F

2024-10-20 06:14:54.406 | WARNING  | lean_dojo.interaction.dojo:__init__:156 - Using Lean 4 without a hard timeout may hang indefinitely.
2024-10-20 06:14:54.407 | INFO     | lean_dojo.interaction.dojo:__enter__:168 - Initializing Dojo for Theorem(repo=LeanGitRepo(url='https://github.com/yangky11/miniF2F-lean4', commit='6bcf0b4940fbf17a1ba83db4ed639fbcb26b1a27'), file_path=PosixPath('MiniF2F/Valid.lean'), full_name='mathd_numbertheory_84')
2024-10-20 06:14:54.407 | WARNING  | lean_dojo.interaction.dojo:__init__:156 - Using Lean 4 without a hard timeout may hang indefinitely.
2024-10-20 06:14:54.407 | INFO     | lean_dojo.interaction.dojo:__enter__:168 - Initializing Dojo for Theorem(repo=LeanGitRepo(url='https://github.com/yangky11/miniF2F-lean4', commit='6bcf0b4940fbf17a1ba83db4ed639fbcb26b1a27'), file_path=PosixPath('MiniF2F/Valid.lean'), full_name='mathd_numbertheory_412')
2024-10-20 06:14:54.407 | WARNING  | lean_dojo.interaction.dojo:__init__:156 - Using Lean 4 without a hard time

[Errno 2] No such file or directory
MiniF2F/Valid.lean mathd_numbertheory_412
[Errno 2] No such file or directory
MiniF2F/Valid.lean mathd_algebra_181
[Errno 2] No such file or directory
MiniF2F/Valid.lean amc12a_2016_p3
[Errno 2] No such file or directory
MiniF2F/Valid.lean mathd_algebra_247
[Errno 2] No such file or directory
MiniF2F/Valid.lean algebra_sqineq_2unitcircatblt1
[Errno 2] No such file or directory
MiniF2F/Valid.lean mathd_numbertheory_629
[Errno 2] No such file or directory
MiniF2F/Valid.lean amc12a_2017_p2
[Errno 2] No such file or directory
MiniF2F/Valid.lean algebra_amgm_sumasqdivbsqgeqsumbdiva
[Errno 2] No such file or directory
MiniF2F/Valid.lean mathd_numbertheory_202
[Errno 2] No such file or directory
MiniF2F/Valid.lean imo_1979_p1
[Errno 2] No such file or directory
MiniF2F/Valid.lean mathd_algebra_51
[Errno 2] No such file or directory
MiniF2F/Valid.lean mathd_algebra_10
[Errno 2] No such file or directory


In [ ]:
import subprocess

def ping_ip(ip):
    try:
        # Ping the IP address with a single packet
        result = subprocess.run(
            ['ping', '-c', '1', '-W', '1', ip], # For Windows, use ['ping', '-n', '1', '-w', '1000', ip]
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )
        return result.returncode == 0
    except Exception as e:
        print(f"Error pinging {ip}: {e}")
        return False

def check_ip_range(base_ip):
    alive_ips = []
    for i in range(1, 255):  # Check IPs from 192.168.0.1 to 192.168.0.254
        ip = f"{base_ip}.{i}"
        if ping_ip(ip):
            alive_ips.append(ip)
            print(f"{ip} is alive.")
        else:
            print(f"{ip} is not alive.")
    return alive_ips

if __name__ == "__main__":
    base_ip = "192.168.0"
    alive_ips = check_ip_range(base_ip)
    print(f"Alive IPs: {alive_ips}")
